In [ ]:
import blpapi
from datetime import datetime

# Inicializa la sesión de Bloomberg
session = blpapi.Session()
if not session.start():
    print("Failed to start session.")
    exit()

if not session.openService("//blp/refdata"):
    print("Failed to open //blp/refdata")
    exit()

service = session.getService("//blp/refdata")
request = service.createRequest("HistoricalDataRequest")

# Añade los tickers
tickers = ["AAPL US Equity", "MSFT US Equity"]  # Ejemplo de tickers
for ticker in tickers:
    request.append("securities", ticker)

# Añade los campos que deseas solicitar, en este caso, la fecha de ganancias
request.append("fields", "EARNINGS_ANNOUNCEMENT_DT")

# Establece el rango de fechas para la solicitud
request.set("startDate", "20190101")
request.set("endDate", datetime.today().strftime("%Y%m%d"))

# Envía la solicitud
print("Sending Request:", request)
session.sendRequest(request)

# Procesa las respuestas
while True:
    event = session.nextEvent()
    for msg in event:
        print(msg)

    if event.eventType() == blpapi.Event.RESPONSE:
        # Evento de respuesta final, sale del bucle
        break

# Cierra la sesión
session.stop()
